In [19]:
import torch
import torch.nn as nn
from multi_stage import LLM
from prep_data import get_eng_hi_dataset, tokenize
from transformers import GPT2LMHeadModel, MT5Tokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

<h2>Obtain parallel data (EN-HI)</h2>
<h5>Data is in the form of dictionary with 'en' and 'hi' keys corresponding to english and hindi sentences respectively</h5>

In [4]:
data = get_eng_hi_dataset()

Found cached dataset parquet (/home1/tejomay/.cache/huggingface/datasets/cfilt___parquet/cfilt--iitb-english-hindi-911387c6837f8b91/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
len(data['en'])

1658362

In [21]:
model = GPT2LMHeadModel.from_pretrained("THUMT/mGPT")

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

<h2>Hyperparameters</h2>

In [23]:
len_prefix = 20
lr = 0.001
batch_size = 256
num_epochs = 100

for param in model.parameters():
    param.requires_grad = False

In [ ]:
MT_model = LLM(model, len_prefix).to(device)

<h2>Training</h2>